# Doc de la bibliothèque python construite pour aider à l'exploration lors du datasprint PORTIC 2023

## Objectif de la bibliothèque

- proposer deux clients pour récupérer les données PORTIC et de TOFLIT18, faciles à manipuler et avec une API la plus unifiée possible
- fournir des utils qui offrent des formatages des données adaptés (récupération de tableaux, construction de réseaux), et qui facilitent le croisement des données PORTIC / TOFLIT18

## Information sur les données

Concernant les clients web pour l'accès aux données, vous pourrez également accéder par vos propres moyens à partir des sources existantes :
   

- Voir la doc de l'API PORTIC : https://gitlab.huma-num.fr/portic/porticapi/-/tree/master/porticapi]
- Voir le répertoire des données toflit18 au format csv : https://github.com/medialab/toflit18_data)
- Voir le code de l'API toflit18 : https://github.com/medialab/toflit18)

## Installer la bibliothèque


La bibliothèque est installée localement par le script `install.sh`, mais vous pouvez également l'installer depuis une copie du répertoire github via `pip install -e datalib)`. Ne pas oublier de la réinstaller après un git pull dans le cas où elle aurait changé.
    
## Sommaire des fonctions proposées
 
#### Fonctions principales (donnent accès aux données de base directement utilisables pour l'étude) :

Portic:

- [get_pointcalls](#portic_client.get_pointcalls(year=1787,-kwargs\)) : récupération de pointcalls (filtres disponibles : année, port d'enregistrement, entrée ou sortie dans le port, type de navire, activité déclarée (pêche, transport de marchandises ...), ...)

Toflit :

- [get_flows](#toflit_client.get_flows(start_year=None,-end_year=None,-year=None,-params=None,-kwargs\)) : récupération de flows (1 flow = import / export d'une marchandise ; infos disponibles : import ou export, produit commercé, partenaire commercial, bureau des Fermes ou a été enregistré le flux ...)



#### Fonctions secondaires (plutôt pour aider à paramétrer les fonctions principales, nourrir des fonctions de visualisation, ...)

Utils : 

- [nest_toflit18_flow](#nest_toflit18_flow(flow\)) : organise un flux toflit18 en un ensemble de clés thématiques
- [nest_portic_pointcall](#nest_portic_pointcall(pointcall\)) : organise un pointcall portic en un ensemble de clés thématiques
- [build_cooccurence_graph](#build_cooccurence_graph(data,-key_1,-key_2,-**kwargs\)) : renvoie un graphe networkx de coocurrence entre des dicts (besoin de fournir 2 clés)
- [get_online_csv](#get_online_csv(url\)) : récupère le contenu d'un csv en ligne (style google spreadsheet)
- [get_pointcalls_commodity_purposes_as_toflit_product](#get_pointcalls_commodity_purposes_as_toflit_product(pointcalls,-product_classification="product_simplification"\)) : correspondance de produits Portic / Toflit sur des pointcalls données en argument 
- [get_pointcalls_port_as_toflit_partner](#get_pointcalls_port_as_toflit_partner(pointcalls,-product_classification="partner_simplification"\))  : correspondance de ports d'enregistrement des congés Portic / partenaires Toflit sur des pointcalls données en argument 
- [get_pointcalls_homeport_as_toflit_partner](#get_pointcalls_homeport_as_toflit_partner(pointcalls,-product_classification="partner_simplification"\)) : correspondance de ports d'appartenance des navires Portic / partenaires Toflit sur des pointcalls données en argument 

Portic:

- [get_flows](#portic_client.get_flows(kwargs\)) : récupération de flux (1 flux portic = 1 trajet intermédiaire (escale), souvent décrit par plusieurs pointcalls)
- [get_travels](#portic_client.get_travels(kwargs\)) : récupération de travels (1 travel = ensemble de flux/escales qui s'enchainent ; infos disponibles : point de départ, d'arrivée, distance parcourue, infos sur le navire ...)
- [get_fieldnames](#portic_client.get_fieldnames(\)) : récupère les noms des variables des données et leur description
- [get_departures_details](#portic_client.get_departures_details(lon=None,-lat=None,-radius=None,-kwargs\)) : récupère tous les ports aux alentours d'un point géographique donné
- [get_destinations_aggregated](#portic_client.get_destinations_aggregated(lon=None,-lat=None,-radius=None,-kwargs\)) : retourne une aggrégation des amirautés de départ de voyages qui arrivent dans des points situés dans le voisinage (voir paramètre radius) du point requêté.
- [get_ports](#portic_client.get_ports(srid=None\)) : description de ports et de leur localisation géographique

Toflit :

- [get_flows_per_year](#toflit_client.get_flows_per_year(type,-kwargs\)) : récupère les flux par année par direction ou par type de source
- [get_locations](#toflit_client.get_locations(classification='partner_orthographic',-kwargs\)) : récupère le réseau des lieux (directions et partenaires) et le montant de leurs échanges 
- [get_time_series](#toflit_client.get_time_series(kwargs\)) : récupère des séries temporelles à propos des flux de marchandises
- [get_customs_regions](#toflit_client.get_customs_regions(\)) : récupère toutes les direction des Fermes de la base Toflit18
- [get_product_classifications](#toflit_client.get_product_classifications(\)) : récupère des informations sur une classification de produit toflit (et optionellement sur ses classifications filles)
- [get_partner_classifications](#toflit_client.get_partner_classifications(\)) : récupère des informations sur une classification de partenaires toflit (et optionellement sur ses classifications filles)
- [get_classification_groups](#toflit_client.get_classification_groups(classification="product_simplification",-kwargs\)) : récupère l'ensemble des catégories associées à une certaine classification (donnée en paramètre)
- [get_sources_types](#toflit_client.get_sources_types(\)) : récupère les types de sources disponibles


### Principe généraux d'utilisation du client

Les fonctions Portic et Toflit sont appelées respectivement depuis portic_client et toflit_client.
Les fonctions utils sont appelables directement.

Si des paramètres spécifiques peuvent être utilisés, ils sont décrit dans la doc spécifique à chaque fonction.

# Imports et bootstrapping des clients

In [ ]:
import networkx as nx
import requests
import csv
import json

# liste des imports possibles
from marseillesprint import Portic
from marseillesprint import Toflit
from marseillesprint import nest_toflit18_flow
from marseillesprint import nest_portic_pointcall
from marseillesprint import build_cooccurence_graph
from marseillesprint import get_pointcalls_commodity_purposes_as_toflit_product
from marseillesprint import get_online_csv
from marseillesprint import get_pointcalls_port_as_toflit_partner
from marseillesprint import get_pointcalls_homeport_as_toflit_partner

portic_client = Portic()
toflit_client = Toflit()

# Méthodes pour l'accès à PORTIC
pouvant être appelées depuis `portic_client` 

Paramètres généraux pouvant être appliqués à (presque) toutes ces méthodes :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
|params|liste de chaînes| ('all', ... tous les noms de params) | all | attributs à intégrer dans la réponse |
|format|chaîne| ('json','csv')| json | format de la réponse|
|shortenfields| booléen | (true, false) | false | permet de raccourcir les noms des attributs et donc d'alléger la taille du JSON téléchargé.|
|both_to| booléen | (true,false) | false | permet de récupérer les données concernant l'arrivée d'un voyage en plus des données du départ (données par défaut). |
| date | string format YYYY | (...années) | 1787 | pour filtrer les données sur une année donnée. L'année de la date d'arrivée ou de la date de départ doit commencer par ces 4 digits : 1787 ou  1789 par exemple. Exemple : http://data.portic.fr/api/pointcalls/?format=json&date=1789 |
| zipped | booléen | (true, false) | false | ... |

# portic_client.get_pointcalls(year=1787, kwargs)


Synopsis: récupère une liste de pointcalls selon les filtres et les paramètres spécifés.

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| start_year | <int/string> | | | année de début |
| end_year | <int/string> | | | année de fin |
| year | <int/string> | | | année exacte (l'emporte sur début/fin) |

\+ tous les noms de propriétés des pointcalls pouvant être donnés en paramètres pour faire office de filtres (accepte une valeur unique, ou une liste de valeurs possibles pour que l'objet filtré soit valide - ex `pointcall_admirality='La Rochelle'`)


### Exemple : je veux tous les pointcalls concernant Marseille

In [9]:
from marseillesprint import Portic
portic_client = Portic()


pointcalls_datasprint = portic_client.get_pointcalls(
 pointcall = 'Marseille',
 year="1787"
)


pointcall = pointcalls_datasprint[0] 
# util pour organiser un pointcall en ensembles de clés thématiques
nest_portic_pointcall(pointcall)

KeyError: 'source_number'

## portic_client.get_fieldnames(API='any')


Synopsis:
Récupère les noms des variables Portic et leur description (pratique quand on ne sait plus ce qu'une colonne/propriété signifie)

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| API | <string> | 'pointcalls' | 'travels' | 'any' | 'any' | type de fieldnames à récupérer |


---

### Exemple :

In [ ]:
from marseillesprint import Portic
portic_client = Portic()


portic_client.get_fieldnames()[0:1]

## portic_client.get_flows(kwargs)

Synopsis:

Retourne une liste de flux, c'est-à-dire de voyages liés à des ports spécifiques, soit en y entrant (direction "in"), soit en en sortant (direction "out"), soit en ayant navigué autour (direction "in-out")

**Attention, les flux sont reconstruits d'après plusieurs pointcalls en fonction de leur incertitude, et sont peu complets pour les données du datasprint.** Il vaut mieux raisonner avec les pointcalls par défaut.

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| ports | list(string) | UHGS_id | 4326 | liste des ids de ports à filtrer (séparés par des virgules)

\+ tous les noms de propriétés des flows portic pouvant être donnés en paramètres pour faire office de filtres (accepte une valeur unique, ou une liste de valeurs possibles pour que l'objet filtré soit valide)



### Exemple : tous les flux passant par Bordeaux (A0180923) ou Boulogne sur Mer (A0152606) ; on filtre les résultats sur les localisations de départ et d'arrivée, et la distance entre ces 2 points

In [ ]:
from marseillesprint import Portic
portic_client = Portic()


flows = portic_client.get_flows(
    ports = ['A0180923', 'A0152606'],
    params =['distance_dep_dest', 'departure', 'departure_latitude', 'departure_longitude', 'destination', 'destination_latitude', 'destination_longitude']
)
flows[0:1]

## portic_client.get_travels(kwargs)


Synopsis:

Récupère les données de trajectoires calculées.

#### possibilité de filtrer à une valeur ou une liste de valeurs par attribut
---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|


### Exemple : tous les voyages au départ d'un port rattaché à la direction des Fermes de La Rochelle, dont le port d'attache du navire est Dieppe et Noirmoutier, et son pavillon français, ...

In [ ]:
from marseillesprint import Portic
portic_client = Portic()


portic_client.get_travels(
    departure_action='Out',
    departure_ferme_direction='La Rochelle',
    homeport_toponyme_fr=['Dieppe', 'Noirmoutier'],
    ship_flag_standardized_en='French'
)[0:1]

## portic_client.get_departures_details(lon=None, lat=None, radius=None, kwargs)


Synopsis:

Récupère tous les ports aux alentours d'un point géographique donné.

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| lon | flottant | (minLon,maxLon) | ? | longitude du centre de la zone à requêter|
| lat | flottant | (minLat,maxLat) | ? | latitude du centre de la zone à requêter|
| radius | flottant | (0,?) | 100 ? |  rayon en kilomètres |


### Exemple : coordonnées de tous les ports recensés par Portic dans un rayon de 30km autour du port de La Rochelle 

In [ ]:
from marseillesprint import Portic
portic_client = Portic()

portic_client.get_departures_details(
 lat = 46.166667, # latitude du port de La Rochelle
 lon = -1.15, # longitude du port de La Rochelle
 radius = 30
)[0:1]

## portic_client.get_destinations_aggregated(lon=None, lat=None, radius=None, kwargs)

Synopsis:

Retourne une aggrégation des amirautés de départ de voyages qui arrivent dans des points situés dans le voisinage (voir paramètre radius) du point requêté.

'label': amirauté de départ

'value': nombre de voyages partis de cette amirauté

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| lon | flottant | (minLon,maxLon) | ? | longitude du centre de la zone à requêter|
| lat | flottant | (minLat,maxLat) | ? | latitude du centre de la zone à requêter|
| radius | flottant | (0,?) | 100 ? |  rayon en kilomètres |


### Exemple : amirautés qui envoient des bateaux autour de l'ile de Ré (à 10km à la ronde), et nombre de bateaux envoyés

In [ ]:
from marseillesprint import Portic
portic_client = Portic()

portic_client.get_destinations_aggregated(
 lat = 46.2, # latitude du port de L'Ile de Ré
 lon = -1.416667, # longitude du port de L'Ile de Ré
 radius = 10
)[0:1]

## portic_client.get_ports(srid=None)

 Synopsis:
 Retourne une liste de *ports_points* au format JSON

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
|srid|int|---|---|identifiant de référence spatiale unique associé à un système de coordonnées, une tolérance et une résolution spécifiques|


### Exemple : description du port avec srid 4326 au format JSON

In [ ]:
from marseillesprint import Portic
portic_client = Portic()

resp = portic_client.get_ports(
    srid = 4326
)
print(json.dumps(resp[0:5], indent=2))

# Méthodes pour l'accès à TOFLIT
pouvant être appelées depuis `toflit_client` :

## toflit_client.get_flows(start_year=None, end_year=None, year=None, params=None, kwargs)

Synopsis : 
récupère les flux toflit18 tels que présentés au format CSV, avec possibilités de filtrage avancé.

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| start_year | <int/string> | | None | année de début |
| end_year | <int/string> | | None | année de fin |
| year | <int/string> | | None | année exacte (l'emporte sur fin/début) |
| params | list |  | toutes les colonnes | propriétés à renvoyer |

\+ tous les noms de propriétés des flux pouvant être donnés en paramètres pour faire office de filtres (accepte une valeur unique, ou une liste de valeurs possibles pour que l'objet filtré soit valide)

## Exemple : tous les flows toflit de 1780 à 1790, enregistrés dans la direction des Fermes de La Rochelle, et où le produit commercé est du vin (on filtre le résultat sur certains params seulement)


In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()

flows = toflit_client.get_flows(
    start_year=1780, 
    end_year=1790, 
    customs_region='La Rochelle',
    product_revolutionempire='Vins divers',
    params=['customs_region', 'partner_simplification', 'product_revolutionempire', 'quantity', 'quantity_unit', 'customs_region_origin']
)
flows[0:1]

# util pour organiser un flow en ensembles thématiques (ne fonctionne que sur un flow avec params=all)
# flow = flows[0]
# nest_toflit18_flow(flow)

## toflit_client.get_customs_regions()

Synopsis :

Récupère toutes les customs region (direction des Fermes) de la base

---

### Exemple

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()


toflit_client.get_customs_regions()[0:1]

## toflit_client.get_product_classifications()

Synopsis :

Récupère les classifications de produits

---

### Exemple : on récupère la classification source (commenter la ligne resp['children']=None pour avoir les classifications filles)

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()

resp = toflit_client.get_product_classifications()
# removing children for the sake of logging
# resp['children'] = None
print(json.dumps(resp, indent=2))

## toflit_client.get_partner_classifications()

Synopsis :

Récupère les classifications de partenaires

---

### Exemple : on récupère toute l'arborescence de classifications (décommenter la ligne resp['children']=None pour n'avoir que la classification source)

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()

resp = toflit_client.get_partner_classifications()
# removing children for the sake of logging
resp['children'] = None
print(json.dumps(resp, indent=2))

## toflit_client.get_classification_groups(classification="product_simplification", kwargs)

Synopsis :

Récupère l'ensemble des catégories associées à une classification en particulier (sans le détail des valeurs)

---

### Exemple : récupération des 5 premières catégories de la classification partner grouping

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()

toflit_client.get_classification_groups('partner_grouping')[0:5]

## toflit_client.get_classification_search(classification='product_revolutionempire', kwargs)

Synopsis :

Récupère le détail des groupements associés à une classification en particulier.

---

Paramètre d'URL `classification` : le nom de la classification préfixé par son type (ex. "product_simplification", ou "partner_source")

---

### Exemple : produits sources recouverts par 2 catégories de product orthographic

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()

toflit_client.get_classification_search('product_orthographic')[0:2]

## toflit_client.get_locations(classification='partner_orthographic', kwargs)

Synopsis :

Récupère le réseau des lieux (directions et partenaires) et le montant de leurs échanges

---

Paramètre `classification` : l'id de la classification de partenaire à utiliser

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| dateMin | entier | (années) | ? | année de début |
| dateMax | entier | (années) | ? | année de fin |
| kind | string | ('total', 'import', 'export') | 'total' | type de flux |
| sourceType | string | (types de source) | ? | id du type de source à utiliser |
| product | liste d'objets | (objects avec {{"id","name","value"}) | None | liste des produits à filtrer |
| productClassification | string | (ids de classification) | None | Classification de produit à utiliser pour le filtre |

---

### Exemple 

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()

toflit_client.get_locations(
    classification = 'partner_grouping', 
    dateMin = 1750,
    dateMax = 1760,
    kind = 'total',
    sourceType = 'Best Guess national product x partner'
)[0:1]

## toflit_client.get_flows_per_year(type, kwargs)

Synopsis :

Récupère les flux par année par direction ou par type de source

---

Paramètre `type` : le type de flux 'direction' ou 'sourceType'

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| dateMin | entier | (années) | ? | année de début |
| dateMax | entier | (années) | ? | année de fin |
| kind | string | ('total', 'import', 'export') | 'total' | type de flux |
| sourceType | string | (types de source) | ? | id du type de source à utiliser |
| partner | liste d'objets | (objects avec {"name","id"}) | None | les partenaires commerciaux à prendre en compte (e.g. {name: 'Alsace', id: 'Alsace~partner_orthographic'}) |
| partnerClassification | string | (ids de classification) | None | Classification de partenaire à utiliser pour le filtre |
| product | liste d'objets | (objects avec {{"id","name","value"}) | None | liste des produits à filtrer |
| productClassification | string | (ids de classification) | None | Classification de produit à utiliser pour le filtre |
| direction | chaîne | (noms de direction) | '$all$' | nom de la direction à filtrer |

---

### Exemple

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()

toflit_client.get_flows_per_year(
    type ='sourceType', 
    dateMin = 1750,
    dateMax = 1780,
    sourceType = 'National toutes directions partenaires manquants'
)

## toflit_client.get_time_series(kwargs)

Synopsis :

Récupère des séries temporelles à propos des flux de marchandises

---

Paramètres de requête spécifiques :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| dateMin | entier | (années) | ? | année de début |
| dateMax | entier | (années) | ? | année de fin |
| kind | string | ('total', 'import', 'export') | 'total' | type de flux |
| sourceType | string | (types de source) | ? | id du type de source à utiliser |
| partner | liste d'objets | (objects avec {"name","id"}) | None | les partenaires commerciaux à prendre en compte (e.g. {name: 'Alsace', id: 'Alsace~partner_orthographic'}) |
| partnerClassification | string | (ids de classification) | None | Classification de partenaire à utiliser pour le filtre |
| product | liste d'objets | (objects avec {{"id","name","value"}) | None | liste des produits à filtrer |
| productClassification | string | (ids de classification) | None | Classification de produit à utiliser pour le filtre |
| direction | chaîne | (noms de direction) | '$all$' | nom de la direction à filtrer |

---

### Exemple

In [ ]:
toflit_client.get_time_series(
    direction = 'La_Rochelle'
)[0:2]

## toflit_client.get_sources_types()

Synopsis :

Récupère les types de sources disponibles

---

### Exemple

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()

toflit_client.get_sources_types()

# Utils



## nest_toflit18_flow(flow)


Synopsis:
Cette fonction organise un flux toflit18 tel que reçu depuis les données en csv
  sous la forme d un ensemble de clés thématiques qui contiennent chacun les propriétés correspondantes.

---

### Exemple : 

In [ ]:
from marseillesprint import Toflit
toflit_client = Toflit()

flows = toflit_client.get_flows(
    year=1789, 
    product_orthographic='dentelle de soie'
) 

flow = flows[0] 
nest_toflit18_flow(flow)

## nest_portic_pointcall(pointcall)


Synopsis:
Cette fonction organise un pointcall portic tel que reçu depuis les données en csv
sous la forme d un ensemble de clés thématiques qui contiennent chacun les propriétés correspondantes.

---

### Exemple : 

In [ ]:
from marseillesprint import Portic
from marseillesprint import nest_portic_pointcall
portic_client = Portic()

pointcalls = portic_client.get_pointcalls(
    year=1787
)

pointcall = pointcalls[0] 
nest_portic_pointcall(pointcall)

## build_cooccurence_graph(data, key_1, key_2, **kwargs)
    
Synopsis :
Cette fonction prend un ensemble de dict (data) et deux noms de clés présents dans chaque dict.
Elle renvoie un graphe networkx de coocurrence entre les dicts.

---

Paramètres de requête spécifiques (**kwargs) :

| nom | type | valeurs | défault | description |
|---|---|---|---|---|
| color_1 | code rgb (string?) |  | rgb(0, 255, 0) | couleur des noeuds de type 1 |
| color_2 | code rgb (string?) |  | rgb(255, 0, 0) | couleur des noeuds de type 2 |
| node_min_size | int |  | 1 | taille min des noeuds |
| node_min_size | int |  | 10 | taille max des noeuds |

### Exemple : 

In [ ]:
from marseillesprint import Toflit
from marseillesprint import build_cooccurence_graph
from ipysigma import Sigma

toflit_client = Toflit()
flows = toflit_client.get_flows(
    start_year=1780, 
    end_year=1790, 
    customs_region='La Rochelle'
)

Graph = build_cooccurence_graph(flows, 'partner', 'product')
Sigma(Graph, start_layout=True)

## get_online_csv(url)

Synopsis:
Cette fonction permet de récupérer le contenu d'un csv en ligne.
Pour les google spreadsheets: fichier > publier sur le web > format csv > copier le lien.
  
Bien mettre l'url sous forme de string dans les params

In [ ]:
from marseillesprint import get_online_csv


pointcalls_from_csv = get_online_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSTh5UY2pCjlPQz2WzHhqeKbAqUHzBpetRZbM9C4SzWK4chFZlIOHLU_zVpMtY8n6kNH1k4hIgpXiRE/pub?gid=0&single=true&output=csv')

pointcalls_from_csv[0:1]

## get_pointcalls_commodity_purposes_as_toflit_product(pointcalls, product_classification="product_simplification")

Synopsis :
Cette fonction prend en entrée une liste de pointcalls et un nom de classification de produits ("product_simplification" par défaut).
Elle renvoie en sortie la liste des dict de pointcalls enrichis avec une propriété "commodity_purposes" qui ont les propriétés existantes de PORTIC pour commodity_purpose[2,3,4] + une propriété "commodity_as_toflit qui donne la valeur correspondante dans toflit18.

In [ ]:
from marseillesprint import Portic
from marseillesprint import get_pointcalls_commodity_purposes_as_toflit_product

portic_client = Portic()

pointcalls = portic_client.get_pointcalls(
 year = 1789
)

with_toflit_products = get_pointcalls_commodity_purposes_as_toflit_product(pointcalls, product_classification='product_simplification')

with_toflit_products[0:1]

## get_pointcalls_port_as_toflit_partner(pointcalls, product_classification="partner_simplification")

Synopsis :
 Cette fonction prend en entrée une liste de pointcalls et un nom de classification de partenaire ("partner_simplification" par défaut).
Elle renvoie en sortie la liste des dict de pointcalls enrichis avec une propriété "pointcall_as_toflit_partner".

In [ ]:
from marseillesprint import Portic
from marseillesprint import get_pointcalls_port_as_toflit_partner

portic_client = Portic()

pointcalls = portic_client.get_pointcalls(
 year = 1789
)

get_pointcalls_port_as_toflit_partner(pointcalls, 'partner_grouping')[0:1]

## get_pointcalls_homeport_as_toflit_partner(pointcalls, product_classification="partner_simplification")
  
Synopsis :
Cette fonction prend en entrée une liste de pointcalls et un nom de classification de partenaire ("partner_simplification" par défaut).
Elle renvoie en sortie la liste des dict de pointcalls enrichis avec une propriété "homeport_as_toflit_partner".

In [ ]:
from marseillesprint import Portic
from marseillesprint import get_pointcalls_homeport_as_toflit_partner

portic_client = Portic()

pointcalls = portic_client.get_pointcalls(
 year = 1789
)

get_pointcalls_homeport_as_toflit_partner(pointcalls, 'partner_simplification')[0:1]